In [8]:
from ultralytics import YOLO
import cv2
import os

# === Configuration ===
model_path = r'C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\Pavement_Dataset_Model\train1\weights\best.pt'
input_path = r'C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\WTour_Dataset\test\images'  # can be image, folder, or video
output_dir = r'C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\Pavement_Model_Inference'
save_txt = True
save_img = True

# === Load YOLO model ===
model = YOLO(model_path)

# === Run Inference ===
results = model.predict(
    source=input_path,
    save=save_img,
    save_txt=save_txt,
    save_conf=True,
    project=output_dir,
    name='exp',
    exist_ok=True
)

print(f"✅ Inference complete. Results saved to: {os.path.join(output_dir, 'exp')}")



image 1/990 C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\WTour_Dataset\test\images\10000015_jpg.rf.45cd4e1f3a8f76136cc01074dda38e99.jpg: 640x640 3 roads, 36.3ms
image 2/990 C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\WTour_Dataset\test\images\10000042_jpg.rf.7f3c5d31a13fa8b504e72dd54d2554da.jpg: 640x640 4 sidewalks, 7 roads, 36.5ms
image 3/990 C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\WTour_Dataset\test\images\10000049_jpg.rf.7d56336b38ddf31b11fb2b51f579e4e2.jpg: 640x640 1 sidewalk, 39.7ms
image 4/990 C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\WTour_Dataset\test\images\10000054_jpg.rf.d1af98cf95e6844ce9bc18af7a5db6e7.jpg: 640x640 2 sidewalks, 38.6ms
image 5/990 C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\WTour_Dataset\test\images\10000074_jpg.rf.a081398ded072355c065ca7d3c26d955.jpg: 640x640 3 sidewalks, 38.3ms
image 6/990 C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalke

KeyboardInterrupt: 

In [10]:
import os
import random
import shutil
import yaml
from tqdm import tqdm

def create_yolo_split(
    input_base_dir, # e.g., C:\augmented_dataset
    output_yolo_dataset_dir, # e.g., C:\yolo_dataset_split
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    class_names=None # List of your class names, e.g., ['pavement', 'obstacle']
):
    """
    Splits a dataset (images and YOLO annotations) into train, val, and test sets
    and organizes them in YOLO format.

    Args:
        input_base_dir (str): Path to the base directory of your combined dataset,
                              expected to contain 'images' and 'annotations' subfolders.
        output_yolo_dataset_dir (str): Path to the desired output directory for the YOLO split.
        train_ratio (float): Proportion of data for the training set.
        val_ratio (float): Proportion of data for the validation set.
        test_ratio (float): Proportion of data for the test set.
        class_names (list): A list of class names in order of their IDs. Required for dataset.yaml.
    """

    if not (abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6):
        raise ValueError("Train, validation, and test ratios must sum to 1.0")
    if class_names is None:
        raise ValueError("Class names must be provided for the dataset.yaml file.")

    input_images_dir = os.path.join(input_base_dir, 'images')
    input_annotations_dir = os.path.join(input_base_dir, 'annotations') # Use 'annotations' as per your structure

    if not os.path.exists(input_images_dir):
        print(f"Error: Input images directory not found at {input_images_dir}")
        return
    if not os.path.exists(input_annotations_dir):
        print(f"Error: Input annotations directory not found at {input_annotations_dir}")
        return

    # Create output directories
    os.makedirs(output_yolo_dataset_dir, exist_ok=True)
    os.makedirs(os.path.join(output_yolo_dataset_dir, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_yolo_dataset_dir, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_yolo_dataset_dir, 'images', 'test'), exist_ok=True)
    os.makedirs(os.path.join(output_yolo_dataset_dir, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_yolo_dataset_dir, 'labels', 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_yolo_dataset_dir, 'labels', 'test'), exist_ok=True)

    image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    random.shuffle(image_files) # Shuffle to ensure random distribution

    total_files = len(image_files)
    if total_files == 0:
        print(f"No image files found in {input_images_dir}. Exiting.")
        return

    num_train = int(total_files * train_ratio)
    num_val = int(total_files * val_ratio)
    num_test = total_files - num_train - num_val # Remaining files go to test to handle rounding

    train_files = image_files[:num_train]
    val_files = image_files[num_train : num_train + num_val]
    test_files = image_files[num_train + num_val :]

    splits = {
        'train': train_files,
        'val': val_files,
        'test': test_files
    }

    print(f"Total files: {total_files}")
    print(f"Train files: {len(train_files)}")
    print(f"Validation files: {len(val_files)}")
    print(f"Test files: {len(test_files)}")

    # Copy files to their respective directories
    for split_name, file_list in splits.items():
        print(f"\nCopying {split_name} files...")
        for img_filename in tqdm(file_list, desc=f"Copying {split_name} images/labels"):
            base_name, _ = os.path.splitext(img_filename)
            
            # Source paths
            src_image_path = os.path.join(input_images_dir, img_filename)
            src_annotation_path = os.path.join(input_annotations_dir, base_name + '.txt')

            # Destination paths
            dest_image_path = os.path.join(output_yolo_dataset_dir, 'images', split_name, img_filename)
            dest_annotation_path = os.path.join(output_yolo_dataset_dir, 'labels', split_name, base_name + '.txt')

            # Copy image
            shutil.copyfile(src_image_path, dest_image_path)
            
            # Copy annotation if it exists
            if os.path.exists(src_annotation_path):
                shutil.copyfile(src_annotation_path, dest_annotation_path)
            else:
                print(f"Warning: Annotation file not found for {img_filename} at {src_annotation_path}. Skipping.")


    # Create dataset.yaml file
    dataset_yaml_content = {
        'path': os.path.abspath(output_yolo_dataset_dir), # Absolute path to the dataset root
        'train': '../images/train',
        'val': '../images/val',
        'test': '../images/test', # Optional, useful for final evaluation
        'nc': len(class_names),
        'names': class_names
    }

    yaml_file_path = os.path.join(output_yolo_dataset_dir, 'dataset.yaml')
    with open(yaml_file_path, 'w') as f:
        yaml.dump(dataset_yaml_content, f, default_flow_style=False)

    print(f"\nDataset split complete!")
    print(f"YOLO format dataset created at: {output_yolo_dataset_dir}")
    print(f"Configuration file generated: {yaml_file_path}")
    print("\nNext steps: You can now point your YOLO training script to this dataset.yaml file.")


if __name__ == "__main__":
    # --- IMPORTANT: Configure your directories and classes here ---
    
    # This should be the base directory where your *combined* images and annotations are.
    # If you followed the previous step, this would be your 'augmented_dataset' folder.
    input_dataset_base = r'C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\data\augmented_dataset_annotations' 
    
    # This will be the new directory where the YOLO-formatted split dataset will be created.
    output_yolo_dataset_base = r'C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\data\augmented_yolo_dataset'

    # !!! IMPORTANT !!! Define your class names in the correct order of their IDs
    # E.g., if class ID 0 is 'sidewalk' and ID 1 is 'road' based on your annotation files
    my_class_names = ['pavement', 'road'] # <<< REPLACE WITH YOUR ACTUAL CLASS NAMES

    # Define split ratios
    train_ratio = 0.8
    val_ratio = 0.15 # Using 15% for validation
    test_ratio = 0.05 # Using 5% for testing (adjust as needed)

    create_yolo_split(
        input_dataset_base,
        output_yolo_dataset_base,
        train_ratio,
        val_ratio,
        test_ratio,
        my_class_names
    )

Total files: 3500
Train files: 2800
Validation files: 525
Test files: 175

Copying train files...


Copying train images/labels: 100%|██████████| 2800/2800 [00:25<00:00, 110.54it/s]



Copying val files...


Copying val images/labels: 100%|██████████| 525/525 [00:04<00:00, 115.06it/s]



Copying test files...


Copying test images/labels: 100%|██████████| 175/175 [00:01<00:00, 135.63it/s]


Dataset split complete!
YOLO format dataset created at: C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\data\augmented_yolo_dataset
Configuration file generated: C:\Users\aswin\OneDrive\Desktop\Sandbox\AudioPavementWalkerPaper\data\augmented_yolo_dataset\dataset.yaml

Next steps: You can now point your YOLO training script to this dataset.yaml file.


In [2]:
from datasets import load_dataset

ds = load_dataset("shawshankvkt/Walking_Tours")

c:\Users\aswin\miniconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\aswin\miniconda3\envs\py38\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aswin\.cache\huggingface\hub\datasets--shawshankvkt--Walking_Tours. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activa

In [6]:
# Show dataset structure
print(ds)

# Show available splits
print(ds.keys())

# Show column names for the 'train' split (if available)
print(ds['train'].column_names)

# Display the first example from the 'train' split
print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 3
    })
})
dict_keys(['train'])
['image']
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1354x761 at 0x2AB8BB4A250>}
